In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind #testet
from scipy.stats import f_oneway #ANOVA
from scipy.stats import chi2_contingency #Teste qui-quadrado
from scipy.stats import t #Intervalo de confiança
from scipy.stats import shapiro # Teste de normalidade de Shapiro-Wilk

In [ ]:

# Leitura CSV
df_AB = pd.read_csv("experimento_teste_ab.csv")

#Imprimir na tela o dataset
display(df_AB.head())

,Visitante_ID,Versão_Página,Visualizações,Conversões
0,1,A,167,1
1,2,A,160,0
2,3,A,170,0
3,4,A,161,1
4,5,A,104,0


In [ ]:
#Informações DataFrame

total_linhas = df_AB.shape[0]

total_colunas = df_AB.shape[1]

tipos_de_variaveis = df_AB.dtypes

estat_descrit = df_AB.describe()


print(f"Total de linhas do DataFrame:{total_linhas} \n")

print(f"Total de Colunas do DataFrame:{total_colunas} \n")

print(f"Tipos das variáveis do DataFrame:\n{tipos_de_variaveis} \n")

print(f"Estatística Descritiva do DataFrame: \n{estat_descrit}")


Total de linhas do DataFrame:2000 

Total de Colunas do DataFrame:4 

Tipos das variáveis do DataFrame:
Visitante_ID      int64
Versão_Página    object
Visualizações     int64
Conversões        int64
dtype: object 

Estatística Descritiva do DataFrame: 
       Visitante_ID  Visualizações   Conversões
count   2000.000000    2000.000000  2000.000000
mean    1000.500000     150.755500     0.115000
std      577.494589      28.809421     0.319102
min        1.000000     100.000000     0.000000
25%      500.750000     126.000000     0.000000
50%     1000.500000     151.000000     0.000000
75%     1500.250000     175.000000     0.000000
max     2000.000000     199.000000     1.000000


##**Questão 1**

In [ ]:
#o valor 1 para indicar que houve uma conversão e o valor 0 para indicar que não houve conversão.
amostra= df_AB['Conversões'].count()
cov_A=  ((df_AB['Conversões'] == 1)  & (df_AB['Versão_Página'] == "A")).sum()
cov_B=  ((df_AB['Conversões'] == 1) & (df_AB['Versão_Página'] == "B")).sum()

# Taxa de conversão
taxa_conv_A = cov_A / amostra
taxa_conv_B = cov_B / amostra

print(f'O cenário que tem a maior taxa de conversão é o cenário B. Com taxa de conversão de: {taxa_conv_B}')


O cenário que tem a maior taxa de conversão é o cenário B. Com taxa de conversão de: 0.0625


##**Questão 2**

In [ ]:

alpha = 0.05         # Nível de significância
beta = 0.20          # Probabilidade de erro do tipo II (1 - poder do teste) | poder do teste normalmente é 0.8
p1 = taxa_conv_A     # Taxa de conversão do cenário A
p2 = 0.10            # Taxa de conversão esperada para o cenário B

# Calculando os valores críticos
Z_alpha = stats.norm.ppf(1 - alpha/2)
Z_beta = stats.norm.ppf(1 - beta)

# Calculando o tamanho da amostra
n = ((Z_alpha + Z_beta)**2 * (p1 * (1 - p1) + p2 * (1 - p2))) / ((p2 - p1)**2)

print(f'O tamanho da amostra necessária para o desenvolvimento de um teste A/B: {round(n)}')




O tamanho da amostra necessária para o desenvolvimento de um teste A/B: 486


In [ ]:

# Leitura CSV
df_pacientes = pd.read_csv("pacientes.csv")

df_pacientes.drop(columns=['Unnamed: 0'], inplace=True)

#Imprimir na tela o dataset
display(df_pacientes.head())

,Pressao_Arterial,Idade,Genero,Etnia,Estado_Saude,Nome_Genero,Nome_Etnia,Nome_Estado_Saude
0,142.755809,31,1,1,1,Feminino,Afro-americano,Com condições de saúde adicionais
1,161.878468,35,1,1,0,Feminino,Afro-americano,Saudável
2,161.508259,39,0,3,1,Masculino,Asiático,Com condições de saúde adicionais
3,144.979408,43,1,3,1,Feminino,Asiático,Com condições de saúde adicionais
4,162.502954,55,0,2,1,Masculino,Hispânico,Com condições de saúde adicionais


##**Questão 3**

In [ ]:
indices= ([909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381, 793, 697,  89, 280, 232, 756, 358,  36, 439, 768, 967, 699, 473, 222,  89, 639, 883, 558, 757,  84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362])

idades_indices = df_pacientes.iloc[indices]

base_saudavel = idades_indices.loc[idades_indices['Nome_Estado_Saude'] == 'Saudável']
base_cardiaca = idades_indices.loc[idades_indices['Nome_Estado_Saude'] != 'Saudável']

base_saudavel_idade= base_saudavel['Idade'].tolist()
base_cardiaca_idade= base_cardiaca['Idade'].tolist()

# Teste t para comparar as médias das idades
resultado_teste_t = ttest_ind(base_cardiaca_idade, base_saudavel_idade)

# Valor-p
valor_p = resultado_teste_t.pvalue
print(valor_p)

# Nível de significância
alpha = 0.05

# Resultado do teste
if valor_p < alpha:
    print("Há evidências estatísticas para rejeitar a hipótese nula.")
else:
    print("Não há evidências estatísticas para rejeitar a hipótese nula.")



0.3373545880249039
Não há evidências estatísticas para rejeitar a hipótese nula.


###Como o p-valor é maior que 0.05 não podemos rejeitar H0, e então não há diferença entre os grupos estatisticamente diferentes. As pessoas saudáveis e com doenças cardiácas tem idades próximas.

##**Questão 4**

In [ ]:

base_saudavel = df_pacientes.loc[df_pacientes['Nome_Estado_Saude'] == 'Saudável']
base_cardiaca = df_pacientes.loc[df_pacientes['Nome_Estado_Saude'] != 'Saudável']


mesmas_pessoas = ((set(base_saudavel['Nome_Genero']) == set(base_cardiaca['Nome_Genero'])) &
                 (set(base_saudavel['Nome_Etnia']) == set(base_cardiaca['Nome_Etnia'])) &
                 (set(base_saudavel['Idade']) == set(base_cardiaca['Idade'])) &
                 (set(base_saudavel['Pressao_Arterial']) == set(base_cardiaca['Pressao_Arterial'])))

print(mesmas_pessoas)



False


###As amostras são independentes. Pois, não temos as mesmas pessoas para ambas as amostras. Nesse caso, cada pessoa é atribuída a apenas um grupo (pessoas com condições de saúde adicionais ou pessoas saudáveis), e as características ou medidas de uma pessoa em um grupo não estão relacionadas às características ou medidas de outra pessoa no outro grupo.

##**Questão 5**

In [ ]:
indices_pa = ([690, 894,  67, 201, 364,  19,  60, 319, 588, 643, 855, 623, 530, 174, 105, 693,   6, 462, 973, 607, 811, 346, 354, 966, 943, 372])

pa_indices = df_pacientes.iloc[indices_pa]

base_saudavel = pa_indices.loc[pa_indices['Nome_Estado_Saude'] == 'Com condições de saúde adicionais']
base_cardiaca = pa_indices.loc[pa_indices['Nome_Estado_Saude'] != 'Com condições de saúde adicionais']

base_saudavel_pa= base_saudavel['Pressao_Arterial'].tolist()
base_cardiaca_pa= base_cardiaca['Pressao_Arterial'].tolist()

# Teste t para comparar a pressão arterial média
resultado_teste_t = ttest_ind(base_cardiaca_pa, base_saudavel_pa)

# valor-p
valor_p = resultado_teste_t.pvalue

print(valor_p)

# nível de significância
alpha = 0.06

# Resultado do teste
if valor_p < alpha:
    print("Há evidências estatísticas para rejeitar a hipótese nula.")
else:
    print("Não há evidências estatísticas para rejeitar a hipótese nula.")




0.9359246016248415
Não há evidências estatísticas para rejeitar a hipótese nula.


###Como o p-valor é maior que 0.06 não podemos rejeitar H0, e então não há diferença entre os grupos estatisticamente diferentes. As pessoas saudáveis e com doenças cardiácas tem pressão arterial próximas.

##**Questão 6**

In [ ]:
#Letra a)

base_af = df_pacientes.loc[df_pacientes['Nome_Etnia'] == 'Afro-americano']
base_as = df_pacientes.loc[df_pacientes['Nome_Etnia'] != 'Asiático']
base_hi = df_pacientes.loc[df_pacientes['Nome_Etnia'] == 'Hispânico']
base_ca = df_pacientes.loc[df_pacientes['Nome_Etnia'] != 'Caucasiano']

Base_Et_AF= base_af['Pressao_Arterial'].tolist()
Base_Et_AS= base_as['Pressao_Arterial'].tolist()
Base_Et_HI= base_hi['Pressao_Arterial'].tolist()
Base_Et_CA= base_ca['Pressao_Arterial'].tolist()

# Teste ANOVA
resultado_anova = f_oneway(Base_Et_AF, Base_Et_AS, Base_Et_HI, Base_Et_CA)

# Valor-p
valor_p = resultado_anova.pvalue

# Nível de significância
alpha = 0.05

print(valor_p)

# Resultado do teste
if valor_p < alpha:
    print("Há evidências estatísticas para rejeitar a hipótese nula.")
else:
    print("Não há evidências estatísticas para rejeitar a hipótese nula.")


0.9735411938477544
Não há evidências estatísticas para rejeitar a hipótese nula.


###Como o p-valor é maior que 0.05 não podemos rejeitar H0, e então não há diferença entre os grupos estatisticamente diferentes. A pressão arterial média é a mesma em todos os grupos étnicos.

In [ ]:
#Letra b)

coeficiente_correlacao = df_pacientes['Idade'].corr(df_pacientes['Pressao_Arterial'])
print(coeficiente_correlacao)


-0.021385356934648957


###A correlação é menor que zero, significa que é negativo, isto é, que as variáveis são inversamente relacionadas. Mas essa correlação é tão próxima de zero que não é considerada significativa. Portanto, com base nesse coeficiente de correlação, não parece haver uma associação linear clara entre a idade dos pacientes e sua pressão arterial. Assim, Hipótese Nula (H0): A idade dos pacientes é independente da pressão arterial.








In [ ]:
#Letra c)
print("Não há evidências estatísticas para rejeitar a hipótese nula.")

Não há evidências estatísticas para rejeitar a hipótese nula.


##**Questão 7**

In [ ]:

# Tabela de contingência
tabela_contingencia = pd.crosstab(df_pacientes['Nome_Genero'], (df_pacientes['Nome_Estado_Saude'] == 'Com condições de saúde adicionais'))

# Realizar o teste do qui-quadrado
chi2, p_valor, graus_liberdade, esperado = chi2_contingency(tabela_contingencia)

# Nível de significância
alpha = 0.05

print(valor_p)

# Resultado do teste
if valor_p < alpha:
    print("Há evidências estatísticas para rejeitar a hipótese nula.")
else:
    print("Não há evidências estatísticas para rejeitar a hipótese nula.")


0.9735411938477544
Não há evidências estatísticas para rejeitar a hipótese nula.


###Como o p-valor é maior que 0.05 não podemos rejeitar H0, e então não há diferença entre os grupos estatisticamente diferentes. Não há uma relação entre o sexo e condições de saúde adicionais.

## **Questão 8**

In [ ]:
df_saude_adicional= df_pacientes.loc[df_pacientes['Nome_Estado_Saude'] == 'Com condições de saúde adicionais']

media = df_saude_adicional['Pressao_Arterial'].mean()
desvio_padrao= df_saude_adicional['Pressao_Arterial'].std()

tamanho_amostra = len(df_saude_adicional)

graus_liberdade = tamanho_amostra - 1

nivel_confianca = 0.95

# Valor crítico da distribuição t de Student
valor_critico = t.ppf((1 + nivel_confianca) / 2, graus_liberdade)

erro_padrao_media = desvio_padrao / np.sqrt(tamanho_amostra)

# Intervalo de confiança
limite_inferior = media - valor_critico * erro_padrao_media
limite_superior = media + valor_critico * erro_padrao_media

# Exibindo o intervalo de confiança
print("Intervalo de Confiança para a Média da Pressão Arterial:")
print("Limite Inferior:", round(limite_inferior, 2))
print("Limite Superior:", round(limite_superior,2))

Intervalo de Confiança para a Média da Pressão Arterial:
Limite Inferior: 149.22
Limite Superior: 151.02


##**Questão 9**

In [ ]:
# Teste de Shapiro-Wilk
stat_shapiro, p_valor_shapiro = shapiro(df_pacientes['Pressao_Arterial'])

print( p_valor_shapiro)

# Exibindo o resultado do teste
if p_valor_shapiro < 0.05:
    print("Rejeitamos a hipótese nula. A distribuição da pressão arterial na população não segue uma distribuição normal.")
else:
    print("Não rejeitamos a hipótese nula. A distribuição da pressão arterial na população segue uma distribuição normal.")

0.06068389490246773
Não rejeitamos a hipótese nula. A distribuição da pressão arterial na população segue uma distribuição normal.


### Como o p-valor é maior que 0.05 não podemos rejeitar H0, e então não há diferença entre os grupos estatisticamente diferentes. A distribuição da pressão arterial na população segue uma distribuição normal.